In [1]:
from google.cloud             import storage

## Practice Uploading a regular file from file path

In [7]:
! touch fart_poop.txt
! ls fart_poop.txt

fart_poop.txt


In [8]:
! gsutil ls gs://mushroom-bucket-le-wagon

gs://mushroom-bucket-le-wagon/heroku_mushroom.tgz
gs://mushroom-bucket-le-wagon/mushroom_learning.tgz
gs://mushroom-bucket-le-wagon/api_and_app_10_03_2022/
gs://mushroom-bucket-le-wagon/data/
gs://mushroom-bucket-le-wagon/models/


In [10]:
file_path="fart_poop.txt"
client = storage.Client()
bucket = client.get_bucket('mushroom-bucket-le-wagon')
blob = bucket.blob(file_path)
blob.upload_from_filename(file_path)

In [11]:
! gsutil ls gs://mushroom-bucket-le-wagon

gs://mushroom-bucket-le-wagon/fart_poop.txt
gs://mushroom-bucket-le-wagon/heroku_mushroom.tgz
gs://mushroom-bucket-le-wagon/mushroom_learning.tgz
gs://mushroom-bucket-le-wagon/api_and_app_10_03_2022/
gs://mushroom-bucket-le-wagon/data/
gs://mushroom-bucket-le-wagon/models/


# Practice uploading a tensorflow model to GCS

In [15]:
import tensorflow as tf
from   tensorflow import keras

In [16]:
!pwd
!ls -rtlh ../raw_data/amanita.jpg
! ls -rtlh ../models_from_gcp/model_1_species_vgg16/
! ls -rtlh ../models_from_gcp/model_2_species_vgg16/
! ls -rtlh ../models_from_gcp/model_3_species_vgg16/

/home/danielsptanner/code/DSP-Tan/mushroom_learning/notebooks
-rwxr-xr-x 1 danielsptanner danielsptanner 85K Mar 11 15:38 ../raw_data/amanita.jpg
total 1.2M
-rw-r--r-- 1 danielsptanner danielsptanner  85K Mar 13 13:57 keras_metadata.pb
-rw-r--r-- 1 danielsptanner danielsptanner 562K Mar 13 13:57 saved_model.pb
-rw-r--r-- 1 danielsptanner danielsptanner 562K Mar 13 13:57 models_model_1_species_vgg16
drwxr-xr-x 2 danielsptanner danielsptanner 4.0K Mar 13 13:57 variables
total 656K
-rw-r--r-- 1 danielsptanner danielsptanner  85K Mar 13 13:57 keras_metadata.pb
-rw-r--r-- 1 danielsptanner danielsptanner 562K Mar 13 13:57 saved_model.pb
drwxr-xr-x 2 danielsptanner danielsptanner 4.0K Mar 13 13:57 variables
total 1.2M
-rw-r--r-- 1 danielsptanner danielsptanner  85K Mar 13 13:57 keras_metadata.pb
-rw-r--r-- 1 danielsptanner danielsptanner 562K Mar 13 13:57 saved_model.pb
-rw-r--r-- 1 danielsptanner danielsptanner 562K Mar 13 13:57 models_model_3_species_vgg16
drwxr-xr-x 2 danielsptanner daniel

In [18]:
img_path='../raw_data/amanita.jpg'
model1_path='../models_from_gcp/model_1_species_vgg16/'
model2_path='../models_from_gcp/model_2_species_vgg16/'
model3_path='../models_from_gcp/model_3_species_vgg16/'

In [19]:
model1=keras.models.load_model(model1_path)

2022-06-07 10:00:34.787409: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-07 10:00:34.787476: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-07 10:00:34.787504: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-2): /proc/driver/nvidia/version does not exist
2022-06-07 10:00:34.789121: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
model1

In [21]:
size=(224,224)
batch_size = 32

img = tf.keras.utils.load_img(img_path, target_size=size)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

In [22]:
prediction1 = model1.predict(img_array)

In [23]:
prediction1

array([[1.2826712e-07, 2.1854573e-06, 9.9965596e-01, 6.1959355e-08,
        3.8890176e-09, 2.3032936e-08, 3.9421622e-09, 2.8218472e-04,
        8.3172781e-11, 5.9072918e-05, 3.1869463e-10, 4.1977546e-09,
        3.7646589e-07]], dtype=float32)

In [24]:
model1.save("gs://mushroom-bucket-le-wagon/model_poop_folder")

2022-06-07 10:02:44.297254: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: gs://mushroom-bucket-le-wagon/model_poop_folder/assets


In [26]:
! gsutil ls gs://mushroom-bucket-le-wagon/model_poop_folder 

gs://mushroom-bucket-le-wagon/model_poop_folder/
gs://mushroom-bucket-le-wagon/model_poop_folder/keras_metadata.pb
gs://mushroom-bucket-le-wagon/model_poop_folder/saved_model.pb
gs://mushroom-bucket-le-wagon/model_poop_folder/assets/
gs://mushroom-bucket-le-wagon/model_poop_folder/variables/


# Practice Downloading Model

In [28]:
model2=keras.models.load_model("gs://mushroom-bucket-le-wagon/model_poop_folder")

In [32]:
(model2.predict(img_array)==prediction1).all()

True

Let's see if this works on google cloud AI platform, and let's investigate the memory issues too.